# OSTk Cross-Platform Validation Against GMAT/Orekit (Field Data Generation Scenario 1)

This tutorial demonstrates how to compare OSTk field data generation (such as gravity field or atmosphere "field") against other tools. 
This example will be comparing atmospheric density generated at a sweep of latitudes, longitudes, and altitudes.

In [1]:
import numpy as np
import pandas as pd
import csv
import os
import plotly.graph_objs as go

from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import DateTime
from ostk.physics.time import Duration
from ostk.physics.coordinate.spherical import LLA

from ostk.physics.environment.atmospheric import Earth as EarthAtmosphericModel

from ostk.physics import Environment

## Set up Comparison Files

Define array inputs that can be changed to include/exclude gmat/orekit result comparisons

In [2]:
filenames = ["gmat_physics/scenario001-field-data-generation.csv",
            "orekit_physics/scenario001-field-data-generation.csv",
            ]
comparisons_to_perform = [
                            ("OSTk - GMAT", [0, 1]),
                            ("OSTk - OREKIT", [0, 2]),
                            ("OREKIT - GMAT", [2, 1]),
]

multiplication_factors = [1.0e-9, 1.0]

## Setup Comparison Scenario in OSTk

Define the atmospheric model

In [3]:
atmos_model = EarthAtmosphericModel(EarthAtmosphericModel.Type.Exponential)

### Create a latitude/longitude/altitude grid

In [4]:
# latitudes = range(-80, 80 + 20, 20)
# longitudes = range(-180, 180, 60)
# altitudes = range(300000, 900000 + 100000, 100000)

In [5]:
latitudes = [-80]
longitudes = [-180]
altitudes = list(range(250000, 950000 + 100000, 100000))

In [16]:
# lla_1 = LLA(
#                 Angle.degrees(float(-80)),
#                 Angle.degrees(float(-180)),
#                 Length.meters(550000.0),
#             )

# lla_2 = LLA(
#                 Angle.degrees(float(-80)),
#                 Angle.degrees(float(-180)),
#                 Length.meters(550000.0132418663),
#             )

# dens_1 = atmos_model.get_density_at(lla_1, Instant.J2000())
# dens_2 = atmos_model.get_density_at(lla_2, Instant.J2000())

In [18]:
# (dens_2 - dens_1)/dens_1

-2.074811952979937e-07

In [6]:
ostk_densities = np.zeros(
    (
        len(latitudes),
        len(longitudes),
        len(altitudes),
    )
)

for k, lat in enumerate(latitudes):
    for j, lon in enumerate(longitudes):
        for i, alt in enumerate(altitudes):
            lla = LLA(
                Angle.degrees(float(lat)),
                Angle.degrees(float(lon)),
                Length.meters(float(alt)),
            )

            # Call the density function
            ostk_densities[k][j][i] = atmos_model.get_density_at(lla, Instant.J2000())

ostk_densities = ostk_densities[0][0].tolist()

---

## Process Cross Platform Results 

Read in reference data from CSV file for GMAT and Orekit

In [7]:
all_comparison_densities = [ostk_densities]

for ind, filename in enumerate(filenames):
    with open(f"{os.getcwd()}/data/{filename}") as csvfile:
        reader = csv.DictReader(csvfile)
        comparison_densities = []
        for row in reader:
            density_iter = float(row[reader.fieldnames[4]])
            
            comparison_densities.append(density_iter * multiplication_factors[ind])
            
    all_comparison_densities.append(comparison_densities)

In [8]:
def to_dataframe(alt_ind, comparison_pairing):
    first_tool = comparison_pairing[0]
    second_tool = comparison_pairing[1]
    
    return [
        altitudes[alt_ind],
        all_comparison_densities[first_tool][alt_ind] - all_comparison_densities[second_tool][alt_ind],
        (all_comparison_densities[first_tool][alt_ind] - all_comparison_densities[second_tool][alt_ind]) / all_comparison_densities[first_tool][alt_ind],
    ]

In [9]:
densities_compared = [
    [
        to_dataframe(alt_ind, comparisons_to_perform[comparison_index][-1])
        for alt_ind in range(0, len(altitudes))
    ]
    for comparison_index in range(0, len(all_comparison_densities))
]

In [10]:
densities_compared_df = [
    pd.DataFrame(
        densities_compared[comparison_index],
        columns=["Alt(m)", f"{comparisons_to_perform[comparison_index][0]} Density Error (kg/m^3)", f"{comparisons_to_perform[comparison_index][0]} Relative Density Error (fractional)"],
    )
    for comparison_index in range(0, len(all_comparison_densities))
]

In [11]:
ostk_densities_df = pd.DataFrame(
        [[altitudes[alt], ostk_densities[alt]] for alt in range(0,len(altitudes))],
        columns=["Alt(m)", "OSTk Density Value (kg/m^3)"],
    )

In [12]:
ostk_densities_df

,Alt(m),OSTk Density Value (kg/m^3)
0,250000,7.248000e-11
1,350000,9.518000e-12
2,450000,1.585000e-12
3,550000,3.182782e-13
4,650000,7.249003e-14
5,750000,2.056299e-14
6,850000,7.833689e-15
7,950000,3.979310e-15


In [13]:
densities_compared_df[0]

,Alt(m),OSTk - GMAT Density Error (kg/m^3),OSTk - GMAT Relative Density Error (fractional)
0,250000,2.107249e-17,2.907353e-07
1,350000,2.364739e-18,2.484492e-07
2,450000,3.450441e-19,2.176935e-07
3,550000,6.603675e-20,2.074812e-07
4,650000,1.336263e-20,1.843375e-07
5,750000,3.070961e-21,1.493441e-07
6,850000,8.322607e-22,1.062412e-07
7,950000,2.910449e-22,7.313954e-08


In [14]:
densities_compared_df[1]

,Alt(m),OSTk - OREKIT Density Error (kg/m^3),OSTk - OREKIT Relative Density Error (fractional)
0,250000,-1.112248e-15,-1.534559e-05
1,350000,5.008320e-26,5.261946e-15
2,450000,-1.089986e-18,-6.876880e-07
3,550000,-7.371116e-27,-2.315935e-14
4,650000,-1.842779e-27,-2.542114e-14
5,750000,-2.114147e-28,-1.028132e-14
6,850000,-5.048710e-29,-6.444869e-15
7,950000,-1.735494e-29,-4.361294e-15


In [15]:
densities_compared_df[2]

,Alt(m),OREKIT - GMAT Density Error (kg/m^3),OREKIT - GMAT Relative Density Error (fractional)
0,250000,1.133321e-15,1.563608e-05
1,350000,2.364739e-18,2.484491e-07
2,450000,1.435030e-18,9.053809e-07
3,550000,6.603676e-20,2.074812e-07
4,650000,1.336263e-20,1.843375e-07
5,750000,3.070961e-21,1.493441e-07
6,850000,8.322607e-22,1.062412e-07
7,950000,2.910449e-22,7.313955e-08
